In [77]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

import os
os.chdir('C:\\Users\\rpalem\\Documents\\practice\\heart')

In [78]:
train_values= pd.read_csv('train_values.csv',index_col='patient_id')
train_labels= pd.read_csv('train_labels.csv',index_col='patient_id')
test_values= pd.read_csv('test_values.csv',index_col='patient_id')


In [79]:
train_values.head()

,slope_of_peak_exercise_st_segment,thal,resting_blood_pressure,chest_pain_type,num_major_vessels,fasting_blood_sugar_gt_120_mg_per_dl,resting_ekg_results,serum_cholesterol_mg_per_dl,oldpeak_eq_st_depression,sex,age,max_heart_rate_achieved,exercise_induced_angina
patient_id,,,,,,,,,,,,,
0z64un,1,normal,128,2,0,0,2,308,0.0,1,45,170,0
ryoo3j,2,normal,110,3,0,0,0,214,1.6,0,54,158,0
yt1s1x,1,normal,125,4,3,0,2,304,0.0,1,77,162,1
l2xjde,1,reversible_defect,152,4,0,0,0,223,0.0,1,40,181,0
oyt4ek,3,reversible_defect,178,1,0,0,2,270,4.2,1,59,145,0


In [80]:
train_values.dtypes

slope_of_peak_exercise_st_segment         int64
thal                                     object
resting_blood_pressure                    int64
chest_pain_type                           int64
num_major_vessels                         int64
fasting_blood_sugar_gt_120_mg_per_dl      int64
resting_ekg_results                       int64
serum_cholesterol_mg_per_dl               int64
oldpeak_eq_st_depression                float64
sex                                       int64
age                                       int64
max_heart_rate_achieved                   int64
exercise_induced_angina                   int64
dtype: object

In [81]:
train_values=pd.get_dummies(train_values)
test_values =pd.get_dummies(test_values)

In [82]:
#from sklearn.preprocessing import MinMaxScaler
#scaler = MinMaxScaler()
#train_values = scaler.fit_transform(train_values)

In [83]:
# for preprocessing the data
from sklearn.preprocessing import StandardScaler

# the model
#from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# for combining the preprocess with model training
from sklearn.pipeline import Pipeline

# for optimizing parameters of the pipeline
from sklearn.model_selection import GridSearchCV

In [88]:
pipe = Pipeline(steps=[('scale', StandardScaler()), 
                       ('clf',  RandomForestClassifier(random_state=42))])
pipe

Pipeline(memory=None,
     steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False))])

In [89]:
#param_grid = {'clf__C': [0.0001, 0.001, 0.01, 1, 10], 
#              'clf__penalty': ['l1', 'l2']}
param_grid = { 
    'clf__n_estimators': [90,100,110],
    'clf__max_features': ['auto', 'sqrt', 'log2'], #, 'log2'
    'clf__max_depth' : [3,4,5],
    'clf__criterion' :['gini', 'entropy']
}

gs = GridSearchCV(estimator=pipe, 
                  param_grid=param_grid, 
                  cv=3)

In [90]:
gs.fit(train_values, train_labels.heart_disease_present)

GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease...timators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__n_estimators': [90, 100, 110], 'clf__max_features': ['auto', 'sqrt', 'log2'], 'clf__max_depth': [3, 4, 5, 6], 'clf__criterion': ['gini', 'entropy']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [91]:
from sklearn.metrics import log_loss

predictions = gs.predict_proba(train_values)
log_loss(train_labels.heart_disease_present, predictions)

0.2986710470914178

## best params model

In [93]:
gs.best_params_

{'clf__criterion': 'entropy',
 'clf__max_depth': 4,
 'clf__max_features': 'auto',
 'clf__n_estimators': 90}

In [94]:
best_grid=gs.best_estimator_
best_grid

Pipeline(memory=None,
     steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=90, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False))])

In [95]:
pipe1 = Pipeline(steps=[('scale', StandardScaler()), 
                       ('clf',  RandomForestClassifier())])
pipe1

Pipeline(memory=None,
     steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
      ...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [96]:
param_grid1 = { 
    'clf__n_estimators': [70,80,90],
    'clf__max_features': ['auto'], #, 'log2'
    'clf__max_depth' : [3,4,5],
    'clf__criterion' :['gini', 'entropy']
}

gs1 = GridSearchCV(estimator=pipe1, 
                  param_grid=param_grid1, 
                  cv=3)

In [97]:
gs1.fit(train_values, train_labels.heart_disease_present)

GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
      ...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__n_estimators': [70, 80, 90], 'clf__max_features': ['auto'], 'clf__max_depth': [3, 4, 5], 'clf__criterion': ['gini', 'entropy']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [98]:
gs1.best_params_

{'clf__criterion': 'entropy',
 'clf__max_depth': 5,
 'clf__max_features': 'auto',
 'clf__n_estimators': 70}

In [99]:
from sklearn.metrics import log_loss

predictions = gs.predict_proba(train_values)
log_loss(train_labels.heart_disease_present, predictions)

0.2986710470914178

## predict submition

In [43]:
predictions = gs.predict_proba(test_values)[:, 1]

In [44]:
submission_format = pd.read_csv('submission_format.csv', index_col='patient_id')

In [45]:
my_submission = pd.DataFrame(data=predictions,
                             columns=submission_format.columns,
                             index=submission_format.index)

In [46]:
my_submission.head()

,heart_disease_present
patient_id,
olalu7,0.398163
z9n6mx,0.166417
5k4413,0.888137
mrg7q5,0.187539
uki4do,0.765771


In [47]:
my_submission.to_csv('submission4.csv')